# Transcrição com Diarização de Fala
Este notebook realiza a transcrição automática de áudio com identificação de diferentes locutores (diarização), utilizando modelos como Whisper e ferramentas como PyAnnote.

In [ ]:
# Transcrição com Diarização (Whisper + Pyannote)

In [ ]:
## Instalação das dependências

## 1. Instalação de dependências
Instala os pacotes necessários para transcrição e diarização, como `faster-whisper` e `pyannote-audio`.

In [ ]:
!pip install openai-whisper pyannote.audio torch librosa ffmpeg-python

In [ ]:
import whisper
from pyannote.audio import Pipeline
from datetime import timedelta
import textwrap

In [ ]:
# === CONFIGURAÇÕES ===

In [ ]:
!ffmpeg -i  -ar 16000 -ac 1 saida.wav  #coloque o caminho do vídeo após o -i

## 2. Carregamento do modelo
Aqui é feito o download e carregamento do modelo de transcrição.

In [ ]:
AUDIO_PATH = ""  # Substitua pelo caminho do seu áudio

In [ ]:
HUGGINGFACE_TOKEN = ""  # Gere em: https://huggingface.co/settings/tokens

In [ ]:
## Transcrição com Whisper

In [ ]:
print("Transcrevendo com Whisper...")
model = whisper.load_model("turbo") #coloque o modelo que voce preferir, recomendado o turbo

## 3. Carregamento e pré-processamento do áudio
Esta seção faz o upload ou carregamento do arquivo de áudio e sua preparação para análise.

In [ ]:
result = model.transcribe(AUDIO_PATH, verbose=False)

In [ ]:
transcribed_text = result['text']
print("Texto transcrito:", transcribed_text)

In [ ]:
segments = result["segments"]

In [ ]:
## Diarização com Pyannote

## 4. Diarização de locutores
Detecta automaticamente os diferentes locutores presentes no áudio.

In [ ]:
print("Detectando falantes com pyannote...")
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token="") #insira o token do huggingface

In [ ]:
diarization = pipeline(AUDIO_PATH)

In [ ]:
## Combinação das falas com os falantes

In [ ]:
def sec_to_timestamp(sec):
  return str(timedelta(seconds=int(sec)))

In [ ]:
final_text = ""
for segment in segments:
    start = segment["start"]
    end = segment["end"]
    text = segment["text"].strip()


    speaker_label = "Speaker ?"


    for turn, _, speaker in diarization.itertracks(yield_label=True):
        if turn.start <= start <= turn.end or turn.start <= end <= turn.end:
            speaker_label = speaker
            break


    final_text += f"{speaker_label}: {text}\n"


print(final_text)

## 5. Transcrição com alinhamento por locutor
Cada trecho de fala é transcrito e associado ao respectivo locutor.

In [ ]:
## Salvar o resultado

In [ ]:
with open("transcricao_com_falantes.txt", "w", encoding="utf-8") as f:
  f.write(final_text)

In [ ]:
print("Arquivo salvo como 'transcricao_com_falantes.txt'")

## 6. Salvamento ou exibição dos resultados
Aqui são exibidos ou exportados os resultados finais da transcrição com diarização.